In [14]:
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from rfdetr import RFDETRBase, RFDETRLarge
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

import os
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Set GPU

In [15]:
# Path to CSV
result_df_path = "rfdetr_large_results.csv"

# Checkpoint
checkpoint = "runs/swinir_haryana_to_test_bihar/checkpoint0099.pth"

# Name of the experiment
experiment = "Swinir Haryana to Swinir Test Bihar"

# Dataset dir
dataset_dir = "data/swinir_haryana_to_test_bihar"

# Model
model_name = "RFDETRLarge Epoch 99"


# separate the source and target states from the experiment name
train_region, test_region = experiment.split(" to ")
print(f"Train region: {train_region}, Test region: {test_region}")


Train region: Swinir Haryana, Test region: Swinir Test Bihar


In [16]:
sv_dataset = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_dir}/test",
    annotations_path=f"{dataset_dir}/test/_annotations.coco.json",
)
print(f"Loaded Dataset: {len(sv_dataset)} images")

Loaded Dataset: 687 images


In [17]:
# model = RFDETRBase(pretrain_weights=checkpoint)
model = RFDETRLarge(pretrain_weights=checkpoint)

targets = []
predictions = []

for path, image, annotations in tqdm(sv_dataset):
    image = Image.open(path)
    detections = model.predict(image, threshold=0.5)

    targets.append(annotations)
    predictions.append(detections)

print(len(targets), len(predictions))

Loading pretrain weights


num_classes mismatch: pretrain weights has 2 classes, but your model has 90 classes
reinitializing detection head with 2 classes
100%|██████████| 687/687 [01:44<00:00,  6.57it/s]

687 687


## mAP Calculation

In [18]:
## mAP calculation (non-class agnostic)
print (100 * "=")
print("Class-specific mAP")
print (100 * "=")
map_metric = MeanAveragePrecision(class_agnostic=False)
map_result = map_metric.update(predictions, targets).compute()
# map_result.plot()

matched_classes=map_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")

# Extract mAP values
mAP_50_95 = map_result.map50_95  # mAP 50:95
mAP_50 = map_result.map50  # mAP 50
mAP_75 = map_result.map75  # mAP 75
print(f"mAP 50:95: {mAP_50_95}, mAP 50: {mAP_50}, mAP 75: {mAP_75}")

# Extract class-wise mAP values
num_classes=3
final_class_wise_mAP = [0]*num_classes
class_wise_mAP=map_result.ap_per_class[:,0].tolist()
for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP

# Compute class-agnostic mAP
print (100 * "=")
print("Class-agnostic mAP")
print (100 * "=")
mAP_metric_agnostic = MeanAveragePrecision(class_agnostic=True)
mAP_result_agnostic = mAP_metric_agnostic.update(predictions, targets).compute()
# Extract class-agnostic mAP values
mAP_50_95_agnostic = mAP_result_agnostic.map50_95  # mAP 50:95
mAP_50_agnostic = mAP_result_agnostic.map50  # mAP 50
mAP_75_agnostic = mAP_result_agnostic.map75  # mAP 75
print(f"CA mAP 50:95: {mAP_50_95_agnostic}, CA mAP 50: {mAP_50_agnostic}, CA mAP 75: {mAP_75_agnostic}")

Class-specific mAP
Matched classes: [0, 1, 2]
mAP 50:95: 0.05384427571506219, mAP 50: 0.11512098208720452, mAP 75: 0.0374456595663156
cls: 0, mAP: 0.0
cls: 1, mAP: 0.0
cls: 2, mAP: 0.34536294626161357
Class-agnostic mAP
CA mAP 50:95: 0.15784880083023808, CA mAP 50: 0.3399807701046479, CA mAP 75: 0.11033833923327223


In [19]:
columns=["Model","Train", "Test", "Images", "CFCBK", "FCBK", "Zigzag", "Class-agnostic AP"]
result_df = pd.DataFrame(columns=columns)
# new_row = [experiment] + final_class_wise_mAP + [mAP_50_agnostic]
new_row = [model_name, train_region, test_region, len(sv_dataset)] + final_class_wise_mAP + [mAP_50_agnostic]
    
result_df.loc[len(result_df)] = new_row  # Using loc to add the row

# Display DataFrame
display(result_df.style.hide(axis="index"))

Model,Train,Test,Images,CFCBK,FCBK,Zigzag,Class-agnostic AP
RFDETRLarge Epoch 99,Swinir Haryana,Swinir Test Bihar,687,0.000000,0.000000,0.345363,0.339981


In [20]:
# result_df.to_csv(result_df_path, index=False)

In [21]:
# Save DataFrame    
result=pd.read_csv(result_df_path)
result=result.reset_index(drop=True)
result = pd.concat([result, result_df], ignore_index=True)
display(result.style.hide(axis="index"))
result.to_csv(result_df_path, index=False)

Model,Train,Test,Images,CFCBK,FCBK,Zigzag,Class-agnostic AP
RFDETRLarge Epoch 49,Bihar,Test Bihar,687,0.283475,0.008090,0.546224,0.520724
RFDETRLarge Epoch 39,Bihar,Test Bihar,687,0.178689,0.003630,0.538959,0.510486
RFDETRLarge Best Total,Swinir Bihar,Swinir Test Bihar,687,0.099010,0.017024,0.576877,0.550956
RFDETRLarge Epoch 49,Swinir Bihar,Swinir Test Bihar,687,0.141914,0.014307,0.550687,0.532579
RFDETRLarge Best Total,Thera Bihar,Thera Test Bihar,687,0.108911,0.000000,0.568649,0.542478
RFDETRLarge Epoch 49,Thera Bihar,Thera Test Bihar,687,0.072607,0.006797,0.590218,0.548028
RFDETRLarge Best Regular,Haryana,Test Bihar,687,0.000000,0.009901,0.469247,0.445401
RFDETRLarge Epoch 49,Haryana,Test Bihar,687,0.000000,0.000683,0.334692,0.341268
RFDETRLarge Epoch 79,Haryana,Test Bihar,687,0.000000,0.000000,0.375667,0.380945
RFDETRLarge Epoch 89,Haryana,Test Bihar,687,0.000000,0.000861,0.371145,0.374380


In [23]:
result=pd.read_csv(result_df_path)
display(result.style.hide(axis="index"))

Model,Train,Test,Images,CFCBK,FCBK,Zigzag,Class-agnostic AP
RFDETRLarge Epoch 49,Bihar,Test Bihar,687,0.283475,0.008090,0.546224,0.520724
RFDETRLarge Epoch 39,Bihar,Test Bihar,687,0.178689,0.003630,0.538959,0.510486
RFDETRLarge Best Total,Swinir Bihar,Swinir Test Bihar,687,0.099010,0.017024,0.576877,0.550956
RFDETRLarge Epoch 49,Swinir Bihar,Swinir Test Bihar,687,0.141914,0.014307,0.550687,0.532579
RFDETRLarge Best Total,Thera Bihar,Thera Test Bihar,687,0.108911,0.000000,0.568649,0.542478
RFDETRLarge Epoch 49,Thera Bihar,Thera Test Bihar,687,0.072607,0.006797,0.590218,0.548028
RFDETRLarge Best Regular,Haryana,Test Bihar,687,0.000000,0.009901,0.469247,0.445401
RFDETRLarge Epoch 49,Haryana,Test Bihar,687,0.000000,0.000683,0.334692,0.341268
RFDETRLarge Epoch 79,Haryana,Test Bihar,687,0.000000,0.000000,0.375667,0.380945
RFDETRLarge Epoch 89,Haryana,Test Bihar,687,0.000000,0.000861,0.371145,0.374380


In [25]:
# result=pd.read_csv(result_df_path)
# result = result.iloc[:, [1, 2, -1]]
# display(result.style.hide(axis="index"))

### Confusion Matrix


In [27]:
# confusion_matrix = sv.ConfusionMatrix.from_detections(
#     predictions=predictions,
#     targets=targets,
#     classes=sv_dataset.classes,
# )

# confusion_matrix.plot(
#     fig_size=(8,8),
#     title=f"Confusion Matrix for {experiment}",
# )
# plt.show()
# plt.close()